In [1]:
import numpy as np 
import cv2
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [1]:
import os
from PIL import Image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# Path to your training data directory
train_dir = r'C:\Users\vshas\OneDrive\Desktop\Major\DATASET\TRAIN'

# Check image integrity
print("Checking image integrity in training data...")
for root, dirs, files in os.walk(train_dir):
    for file in files:
        try:
            # Attempt to open the image file
            image_path = os.path.join(root, file)
            img = Image.open(image_path)
            img.load()
        except (OSError, IOError) as e:
            # If an error occurs, print the filename and error message
            print(f"Error opening {image_path}: {e}")

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical'
)

# Model architecture
yoga_model = Sequential()
yoga_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
yoga_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
yoga_model.add(MaxPooling2D(pool_size=(2, 2)))
yoga_model.add(Dropout(0.25))
yoga_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
yoga_model.add(MaxPooling2D(pool_size=(2, 2)))
yoga_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
yoga_model.add(MaxPooling2D(pool_size=(2, 2)))
yoga_model.add(Dropout(0.25))
yoga_model.add(Flatten())
yoga_model.add(Dense(1024, activation='relu'))
yoga_model.add(Dropout(0.5))
yoga_model.add(Dense(5, activation='softmax'))  # Modified to have 5 units

# Model compilation
yoga_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])


Checking image integrity in training data...
Found 1080 images belonging to 5 classes.


In [2]:

# Training
yoga_model_info = yoga_model.fit(
    train_generator,
    steps_per_epoch=(1081 // 64),
    epochs=30,
    validation_data=train_generator,
    validation_steps=(1081 // 64)
)

# Saving model weights
yoga_model.save_weights('yoga_model.h5')



Epoch 1/30
 4/16 [======>.......................] - ETA: 10s - loss: 1.6254 - accuracy: 0.1758

C:\ProgramData\Anaconda3\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


16/16 [==============================] - 30s 2s/step - loss: 1.6031 - accuracy: 0.2274 - val_loss: 1.5913 - val_accuracy: 0.2480
Epoch 2/30
16/16 [==============================] - 31s 2s/step - loss: 1.5878 - accuracy: 0.2411 - val_loss: 1.5888 - val_accuracy: 0.2598
Epoch 3/30
16/16 [==============================] - 29s 2s/step - loss: 1.5911 - accuracy: 0.2667 - val_loss: 1.5867 - val_accuracy: 0.2617
Epoch 4/30
16/16 [==============================] - 29s 2s/step - loss: 1.5854 - accuracy: 0.2657 - val_loss: 1.5757 - val_accuracy: 0.3604
Epoch 5/30
16/16 [==============================] - 29s 2s/step - loss: 1.5663 - accuracy: 0.3022 - val_loss: 1.5553 - val_accuracy: 0.4053
Epoch 6/30
16/16 [==============================] - 28s 2s/step - loss: 1.5384 - accuracy: 0.3691 - val_loss: 1.5129 - val_accuracy: 0.4326
Epoch 7/30
16/16 [==============================] - 29s 2s/step - loss: 1.4756 - accuracy: 0.4134 - val_loss: 1.4030 - val_accuracy: 0.4727
Epoch 8/30
16/16 [=============

In [4]:
yoga_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 46, 46, 32)        320       
                                                                 
 conv2d_5 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 22, 22, 64)        0         
 g2D)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 22, 22, 64)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 20, 20, 128)       73856     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 10, 10, 128)       0         
 g2D)                                                 

In [1]:
import tkinter as tk
from tkinter import *
import cv2
from PIL import Image, ImageTk
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import threading

# Load the trained model for yoga pose classification
yoga_model = Sequential()
yoga_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
yoga_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
yoga_model.add(MaxPooling2D(pool_size=(2, 2)))
yoga_model.add(Dropout(0.25))
yoga_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
yoga_model.add(MaxPooling2D(pool_size=(2, 2)))
yoga_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
yoga_model.add(MaxPooling2D(pool_size=(2, 2)))
yoga_model.add(Dropout(0.25))
yoga_model.add(Flatten())
yoga_model.add(Dense(1024, activation='relu'))
yoga_model.add(Dropout(0.5))
yoga_model.add(Dense(5, activation='softmax'))

yoga_model.load_weights('yoga_model.h5')  # Load pre-trained weights

pose_dict = {0: "downdog", 1: "goddess", 2: "plank", 3: "tree", 4: "warrior2"}

pose_images = {
    "downdog": r"C:\Users\vshas\OneDrive\Desktop\Major\DATASET\op\dd.png",
    "goddess": r"C:\Users\vshas\OneDrive\Desktop\Major\DATASET\op\gd.png",
    "plank": r"C:\Users\vshas\OneDrive\Desktop\Major\DATASET\op\plank.jpg",
    "tree": r"C:\Users\vshas\OneDrive\Desktop\Major\DATASET\op\treee.png",
    "warrior2": r"C:\Users\vshas\OneDrive\Desktop\Major\DATASET\op\w.png"
}

pose_advantages = {
    "downdog": "Stretches the shoulders, hamstrings, calves, arches, and hands. Strengthens the arms and legs. Relieves back pain.",
    "goddess": "Strengthens the legs and core muscles. Opens the hips and chest. Improves posture.",
    "plank": "Strengthens the core muscles, shoulders, arms, and wrists. Improves posture and balance.",
    "tree": "Improves balance and concentration. Strengthens the legs, ankles, and core muscles.",
    "warrior2": "Strengthens the legs, ankles, and arms. Stretches the groins, chest, lungs, and shoulders."
}

# Function to preprocess the frame before prediction
def preprocess_frame(frame):
    processed_frame = cv2.resize(frame, (48, 48))  # Resize frame to match model input size
    processed_frame = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    processed_frame = np.expand_dims(processed_frame, axis=-1)  # Add channel dimension
    processed_frame = np.expand_dims(processed_frame, axis=0)  # Add batch dimension
    processed_frame = processed_frame / 255.0  # Normalize pixel values
    return processed_frame

# Function to process video frames and perform yoga pose recognition
def recognize_pose():
    cap = cv2.VideoCapture(r"C:\Users\vshas\OneDrive\Desktop\Major\DATASET\gd.mp4")  # Capture video from webcam
    if not cap.isOpened():
        print("Can't open camera")
        return
    while True:
        ret, frame = cap.read()  # Read a frame from the video stream
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert frame to RGB
        frame = cv2.resize(frame, (400, 400))  # Resize frame

        # Preprocess the frame for prediction
        processed_frame = preprocess_frame(frame)

        # Perform yoga pose prediction
        prediction = yoga_model.predict(processed_frame)
        max_index = np.argmax(prediction)
        predicted_pose = pose_dict[max_index]

        # Display the frame with the predicted pose label
        display_frame(frame, predicted_pose)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Function to display the frame with the predicted pose label, image, and advantages
def display_frame(frame, predicted_pose):
    frame = Image.fromarray(frame)  # Convert frame to PIL Image
    frame = ImageTk.PhotoImage(image=frame)  # Convert PIL Image to Tkinter PhotoImage
    
    # Display input image and information on the left side
    input_label.config(image=frame)
    input_label.image = frame  # Keep reference to the image object to prevent garbage collection

    # Load and display image of the predicted pose on the right side
    pose_image_path = pose_images.get(predicted_pose, "path/to/default_image.jpg")
    pose_img = Image.open(pose_image_path)
    pose_img = pose_img.resize((400, 400), Image.ANTIALIAS)
    pose_img = ImageTk.PhotoImage(pose_img)
    pose_label.config(image=pose_img)
    pose_label.image = pose_img  # Keep reference to the image object to prevent garbage collection

    # Display pose name and advantages on the right side
    pose_label_text.config(text=f"Predicted Pose: {predicted_pose}\n\nAdvantages:\n{pose_advantages.get(predicted_pose, '')}")

# Initialize Tkinter window
root = tk.Tk()
root.title("Yoga Pose Recognition")
root.geometry("1000x600")
root['bg'] = 'white'

# Create frame for left side (input image and information)
left_frame = tk.Frame(root)
left_frame.pack(side=tk.LEFT, padx=100, pady=10)

# Create frame for right side (output image, pose name, and advantages)
right_frame = tk.Frame(root)
right_frame.pack(side=tk.RIGHT, padx=100, pady=10)


# Create label to display input image
input_label = tk.Label(left_frame)
input_label.pack()

# Create label to display output image
pose_label = tk.Label(right_frame)
pose_label.pack()

# Create label to display predicted pose and advantages
pose_label_text = tk.Label(right_frame, font=("Arial", 12), justify=CENTER)
pose_label_text.pack()

# Start a new thread to run the yoga pose recognition function
threading.Thread(target=recognize_pose).start()

# Start the Tkinter main loop
root.mainloop()


1/1 [==============================] - 0s 31ms/step


MAIN CODE

In [16]:
import tkinter as tk
from tkinter import *
import cv2
from PIL import Image, ImageTk
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import threading

# Load the trained model for yoga pose classification
yoga_model = Sequential()
yoga_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
yoga_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
yoga_model.add(MaxPooling2D(pool_size=(2, 2)))
yoga_model.add(Dropout(0.25))
yoga_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
yoga_model.add(MaxPooling2D(pool_size=(2, 2)))
yoga_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
yoga_model.add(MaxPooling2D(pool_size=(2, 2)))
yoga_model.add(Dropout(0.25))
yoga_model.add(Flatten())
yoga_model.add(Dense(1024, activation='relu'))
yoga_model.add(Dropout(0.5))
yoga_model.add(Dense(5, activation='softmax'))

yoga_model.load_weights('yoga_model.h5')  # Load pre-trained weights

pose_dict = {0: "downdog", 1: "goddess", 2: "plank", 3: "tree", 4: "warrior2"}

pose_images = {
    "downdog": r"C:\Users\vshas\OneDrive\Desktop\Major\DATASET\op\dd.png",
    "goddess": r"C:\Users\vshas\OneDrive\Desktop\Major\DATASET\op\gd.png",
    "plank": r"C:\Users\vshas\OneDrive\Desktop\Major\DATASET\op\plank.jpg",
    "tree": r"C:\Users\vshas\OneDrive\Desktop\Major\DATASET\op\treee.png",
    "warrior2": r"C:\Users\vshas\OneDrive\Desktop\Major\DATASET\op\w.png"
}

pose_advantages = {
    "downdog": "Stretches the shoulders, hamstrings, calves, arches, and hands. Strengthens the arms and legs. Relieves back pain.",
    "goddess": "Strengthens the legs and core muscles. Opens the hips and chest. Improves posture.",
    "plank": "Strengthens the core muscles, shoulders, arms, and wrists. Improves posture and balance.",
    "tree": "Improves balance and concentration. Strengthens the legs, ankles, and core muscles.",
    "warrior2": "Strengthens the legs, ankles, and arms. Stretches the groins, chest, lungs, and shoulders."
}

# Define a dictionary mapping each pose to a set of reference images
reference_images = {
    "downdog": r"C:\Users\vshas\OneDrive\Desktop\Major\DATASET\how-to\how-dd.png",
    "goddess": r"C:\Users\vshas\OneDrive\Desktop\Major\DATASET\how-to\how-g.png",
    "plank": r"C:\Users\vshas\OneDrive\Desktop\Major\DATASET\how-to\how-p.png",
    "tree": r"C:\Users\vshas\OneDrive\Desktop\Major\DATASET\how-to\how-t.png",
    "warrior2": r"C:\Users\vshas\OneDrive\Desktop\Major\DATASET\how-to\how-w.png"
}

# Function to preprocess the frame before prediction
def preprocess_frame(frame):
    processed_frame = cv2.resize(frame, (48, 48))  # Resize frame to match model input size
    processed_frame = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    processed_frame = np.expand_dims(processed_frame, axis=-1)  # Add channel dimension
    processed_frame = np.expand_dims(processed_frame, axis=0)  # Add batch dimension
    processed_frame = processed_frame / 255.0  # Normalize pixel values
    return processed_frame

# Function to suggest the correct pose if the detected pose is improper
def suggest_correct_pose(predicted_pose):
    reference_image_path = reference_images.get(predicted_pose)
    if reference_image_path:
        img = Image.open(reference_image_path)
        img = img.resize((500, 500), Image.ANTIALIAS)
        img = ImageTk.PhotoImage(img)
        reference_label.configure(image=img)
        reference_label.image = img
    else:
        print("Reference image not found for pose:", predicted_pose)

# Function to process video frames and perform yoga pose recognition
def recognize_pose():
    cap = cv2.VideoCapture(r"C:\Users\vshas\OneDrive\Desktop\Major\DATASET\wrr.mp4")  # Video path
    if not cap.isOpened():
        print("Can't open camera")
        return

    def update_frame():
        ret, frame = cap.read()  # Read a frame from the video stream
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert frame to RGB
            frame = cv2.resize(frame, (450, 500))  # Resize frame

            # Preprocess the frame for prediction
            processed_frame = preprocess_frame(frame)

            # Perform yoga pose prediction
            prediction = yoga_model.predict(processed_frame)
            max_index = np.argmax(prediction)
            predicted_pose = pose_dict.get(max_index)
            if predicted_pose:
                display_frame(frame, predicted_pose)
            else:
                print("Unknown pose index:", max_index)

        # Schedule the next update after 10 milliseconds
        root.after(10, update_frame)

    # Start updating the frame
    update_frame()

def display_frame(frame, predicted_pose):
    frame = Image.fromarray(frame)  # Convert frame to PIL Image
    frame = ImageTk.PhotoImage(image=frame)  # Convert PIL Image to Tkinter PhotoImage

    # Display input image and information on the left side
    input_label.configure(image=frame)
    input_label.image = frame

    # Display output image and pose information in the center
    pose_image_path = pose_images.get(predicted_pose)
    if pose_image_path:
        pose_img = Image.open(pose_image_path)
        pose_img = pose_img.resize((400, 400), Image.ANTIALIAS)
        pose_img = ImageTk.PhotoImage(pose_img)
        pose_label.configure(image=pose_img)
        pose_label.image = pose_img
    else:
        print("Image not found for pose:", predicted_pose)

    pose_label_text.config
    
    
        # Update pose label text
    pose_label_text.config(text=f"Predicted Pose: {predicted_pose}\n\nAdvantages:\n{pose_advantages.get(predicted_pose, '')}")

    # Suggest the correct pose if the detected pose is improper
    suggest_correct_pose(predicted_pose)

# Initialize Tkinter window
root = tk.Tk()
root.title("Yoga Pose Recognition")
root.geometry("1200x400")
root['bg'] = 'white'

# Create frame for left side (input image and information)
left_frame = tk.Frame(root)
left_frame.pack(side=tk.LEFT, padx=30, pady=10)

# Create frame for center (output image, pose name, and advantages)
center_frame = tk.Frame(root)
center_frame.pack(side=tk.LEFT, padx=30, pady=10)

# Create frame for right side (reference image)
right_frame = tk.Frame(root)
right_frame.pack(side=tk.LEFT, padx=30, pady=10)

# Create label to display input image
input_label = tk.Label(left_frame)
input_label.pack()

# Create label to display output image
pose_label = tk.Label(center_frame)
pose_label.pack()

# Create label to display predicted pose and advantages
pose_label_text = tk.Label(center_frame, font=("Arial", 12), justify=LEFT, wraplength=400)
pose_label_text.pack()

# Create label to display reference image
reference_label = tk.Label(right_frame)
reference_label.pack()

# Start a new thread to run the recognize_pose function
threading.Thread(target=recognize_pose).start()

# Start the Tkinter main loop
root.mainloop()



1/1 [==============================] - 0s 20ms/step
